In [1]:
import pandas as pd

import random
import numpy as np

In [2]:
pl_df = pd.read_csv('2016_problem_logs_seq.csv')

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
len(pl_df)

10937556

In [4]:
pl_df = pl_df[pl_df['original'] == 1]

In [6]:
len(pl_df['user_id'].unique())

51747

In [7]:
sublist = random.sample(pl_df['user_id'].unique(), 25000)

In [8]:
len(sublist)

25000

In [9]:
sub_df = pd.DataFrame(pl_df[(pl_df['user_id'].isin(sublist))])

In [10]:
len(sub_df)

5033660

In [11]:
len(sub_df['sequence_id'].unique())

63306

In [12]:
sub_counts = pd.value_counts(sub_df['sequence_id'])

len(sub_counts[sub_counts > 400])

2984

In [15]:
len(sub_counts[sub_counts > 700])

1450

In [16]:
mask = sub_df['sequence_id'].isin(sub_counts[sub_counts > 700].index)
sub_df['sequence_id'][~mask] = "-"

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [18]:
sub_df[sub_df['discrete_score'] == 0].head()

,id,assignment_id,user_id,assistment_id,problem_id,original,correct,answer_id,answer_text,first_action,...,actions,tutor_strategy_id,tutor_mode,assignment_type,teacher_comment,network_state,path_info,discrete_score,updated_at,sequence_id
15,176936182,1525286,70190,817748,1197574,1,0.000000,NaN,2,0,...,"--- \n- - start\n - 1452622481273\n - ""43281...",926032.0,tutor,ClassAssignment,NaN,CONNECTED,/PS4328147,0.0,2016-01-12 13:15:59.921411,-
16,176936299,1525286,70190,817749,1197575,1,0.666667,NaN,4,0,...,"--- \n- - start\n - 1452622559286\n - ""43281...",NaN,tutor,ClassAssignment,NaN,CONNECTED,/PS4328147,0.0,2016-01-13 13:45:07.615245,-
19,177160569,1531489,70190,896253,1296260,1,0.000000,NaN,No,0,...,"--- \n- - start\n - 1453151203178\n - ""43488...",NaN,tutor,ClassAssignment,NaN,CONNECTED,/PS4348810/PS4348826,0.0,2016-01-18 16:07:41.249183,-
20,177160581,1531489,70190,896254,1296261,1,0.000000,NaN,2,0,...,"--- \n- - start\n - 1453151260958\n - ""43488...",NaN,tutor,ClassAssignment,NaN,CONNECTED,/PS4348810/PS4348826,0.0,2016-01-18 16:07:49.112993,-
21,177160583,1531489,70190,896255,1296262,1,0.000000,NaN,2,0,...,"--- \n- - start\n - 1453151268827\n - ""43488...",NaN,tutor,ClassAssignment,NaN,CONNECTED,/PS4348810/PS4348826,0.0,2016-01-18 16:07:52.525985,-


In [19]:
sub_df.columns

Index([u'id', u'assignment_id', u'user_id', u'assistment_id', u'problem_id',
       u'original', u'correct', u'answer_id', u'answer_text', u'first_action',
       u'hint_count', u'bottom_hint', u'attempt_count', u'start_time',
       u'end_time', u'first_response_time', u'overlap_time', u'actions',
       u'tutor_strategy_id', u'tutor_mode', u'assignment_type',
       u'teacher_comment', u'network_state', u'path_info', u'discrete_score',
       u'updated_at', u'sequence_id'],
      dtype='object')

In [43]:
new_df = pd.DataFrame()
new_df = pd.get_dummies(sub_df['sequence_id'])
# u'correct , u'bottom_hint, u'hint_count, u'attempt_count, u'first_response_time, first_action
# binary correctness
new_df['correct'] = np.where(sub_df['correct'] < 1, 0, 1)
new_df['bottom_hint'] = sub_df['bottom_hint']
new_df['hint_count'] = sub_df['hint_count']
new_df['attempt_count'] = sub_df['attempt_count']
new_df['first_response_time'] = sub_df['first_response_time']
# first action
new_df['first_action'] = sub_df['first_action']

norm = lambda x: (x - x.min()) / (x.max() - x.min()) if x.max() - x.min() else 1
# normalize within problems
new_df.insert(len(new_df.columns), 'NofFRT', sub_df.groupby('sequence_id')['first_response_time'].transform(norm))
del new_df['first_response_time']

new_df.insert(len(new_df.columns), 'normalized_hint_count', sub_df.groupby('sequence_id')['hint_count'].transform(norm))
del new_df['hint_count']

new_df.insert(len(new_df.columns), 'normalized_attempt_count', sub_df.groupby('sequence_id')['attempt_count'].transform(norm))
del new_df['attempt_count']

new_df['user_id'] = sub_df['user_id']
new_df['id'] = sub_df['id']

In [44]:
new_df.isnull().values.any()

False

In [45]:
new_df.tail()

,2285,2427,2646,2650,5110,5331,5343,5582,5624,5625,...,694733,-,correct,bottom_hint,first_action,NofFRT,normalized_hint_count,normalized_attempt_count,user_id,id
10937535,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0.031843,1.0,0.0,389247,183606214
10937536,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.030833,1.0,0.0,389247,183606230
10937537,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0.039927,1.0,0.0,389247,183606247
10937538,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0.090496,1.0,0.0,389247,183606261
10937539,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.041199,1.0,0.0,389247,183606296


In [46]:
with open('ps_list', 'w') as file:
    for ite in new_df.columns[:963].tolist():
        file.write(str(ite) + "\n")

In [47]:
new_df.to_csv('pl_train_data.csv', index=False)

### Code below is deprecated. Keep it for reference.

In [ ]:
len(pl_df.groupby('user_id').filter(lambda x: len(x) > 100))

In [ ]:

mask = pl_df['sequence_id'].isin(counts[counts > 500].index)
pl_df['sequence_id'][~mask] = "-"

new_df = pd.DataFrame()
new_df = pd.get_dummies(pl_df['sequence_id'])
# u'correct , u'bottom_hint, u'hint_count, u'attempt_count, u'first_response_time
new_df['correct'] = pl_df['correct']
new_df['bottom_hint'] = pl_df['bottom_hint']
new_df['hint_count'] = pl_df['hint_count']
new_df['attempt_count'] = pl_df['attempt_count']
new_df['first_response_time'] = pl_df['first_response_time']
new_df['user_id'] = pl_df['user_id']
new_df['id'] = pl_df['id']

new_df['bottom_hint'].fillna(0, inplace=True)
norm = lambda x: (x - x.min()) / (x.max() - x.min())
new_df.insert(len(new_df.columns), 'NofFRT', pl_df.groupby('sequence_id')['first_response_time'].transform(norm))
del new_df['first_response_time']

new_df['correct'].fillna(0, inplace=True)
new_df['NofFRT'].fillna(1, inplace=True)
new_df['hint_count'].fillna(1, inplace=True)
new_df.to_csv('../LSTM-autoencoder/train_data.csv', index=False)

In [23]:
# convert training data
train_df = pd.read_csv('22_problem_logs_seq.csv')

In [24]:
train_df.head()

,id,assignment_id,user_id,assistment_id,problem_id,original,correct,answer_id,answer_text,first_action,...,actions,tutor_strategy_id,tutor_mode,assignment_type,teacher_comment,network_state,path_info,discrete_score,updated_at,sequence_id
0,83251979,390779,131019,97389,186155,1,1.0,NaN,64,0.0,...,--- \n- - start\n - 1317738211496\n- - answer...,136294.0,tutor,ClassAssignment,NaN,NaN,NaN,NaN,NaN,11833
1,83252617,390779,131019,97411,186221,1,1.0,NaN,65,0.0,...,--- \n- - start\n - 1317738254825\n- - answer...,136338.0,tutor,ClassAssignment,NaN,NaN,NaN,NaN,NaN,11833
2,83253196,390779,131019,97495,186473,1,1.0,NaN,452,0.0,...,--- \n- - start\n - 1317738290513\n- - answer...,136506.0,tutor,ClassAssignment,NaN,NaN,NaN,NaN,NaN,11833
3,83253881,390782,131019,32139,49282,1,0.0,NaN,145,0.0,...,--- \n- - start\n - 1317738340042\n- - answer...,23057.0,tutor,ClassAssignment,NaN,NaN,NaN,NaN,NaN,5924
4,83255489,390782,131019,32120,49263,1,1.0,NaN,199,0.0,...,--- \n- - start\n - 1317738468101\n- - answer...,23038.0,tutor,ClassAssignment,NaN,NaN,NaN,NaN,NaN,5924


In [25]:
# read ps_list
ps_list = []
with open('ps_list') as file:
    for line in file:
        ps_list.append(int(line))

In [41]:
len(ps_list)

1032

In [42]:
encode_list = []
for index, row in train_df.iterrows():
    encode = np.zeros(len(ps_list)+1)
    try:
        idx = ps_list.index(row['sequence_id'])
        encode[idx] = 1
    except ValueError:
        encode[len(ps_list)] = 1

    encode_list.append(encode)
encode_array = np.stack(encode_list, axis=0)

In [43]:
encode_array.shape

(390470, 1033)

In [44]:
final_df = pd.DataFrame(encode_array, columns=ps_list+['-'])

In [48]:
final_df.tail()

,2246,2285,2427,2646,2650,5110,5331,5339,5355,5582,...,694614,694733,-,correct,bottom_hint,hint_count,attempt_count,NofFRT,user_id,id
390465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.000000,0.0,0.0,1,0.000117,346048,182210277
390466,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.000000,0.0,0.0,1,0.000188,346048,182210286
390467,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.333333,0.0,0.0,3,0.000084,346048,182210291
390468,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.000000,0.0,0.0,1,0.000142,346048,182210328
390469,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.000000,0.0,0.0,1,0.000024,346048,182210338


In [47]:
# u'correct , u'bottom_hint, u'hint_count, u'attempt_count, u'first_response_time
final_df['correct'] = train_df['correct']
final_df['bottom_hint'] = train_df['bottom_hint']
final_df['hint_count'] = train_df['hint_count']
final_df['attempt_count'] = train_df['attempt_count']
final_df['first_response_time'] = train_df['first_response_time']

norm = lambda x: (x - x.min()) / (x.max() - x.min())
final_df.insert(len(final_df.columns), 'NofFRT', train_df.groupby('sequence_id')['first_response_time'].transform(norm))
del final_df['first_response_time']

final_df['user_id'] = train_df['user_id']
final_df['id'] = train_df['id']

In [56]:
final_df['correct'].fillna(0, inplace=True)
final_df['NofFRT'].fillna(0.5, inplace=True)
final_df['hint_count'].fillna(1, inplace=True)
final_df['bottom_hint'].fillna(0, inplace=True)
final_df.isnull().values.any()

False

In [57]:
final_df.to_csv('22_train_data.csv', index=False)